In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

In [2]:
df = pd.read_csv('Data/Providers_matched_byNPI_updated.txt', sep='\t', low_memory=False)

In [3]:
#comfirming dataframe seems to have read in correctly
df.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_city,nppes_provider_zip5,...,average_age_of_beneficiaries,beneficiary_female_count,beneficiary_male_count,beneficiary_nondual_count,beneficiary_dual_count,beneficiary_average_risk_score,EXCLYear,REINYear,excl_type,exclusion_flag
0,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,CUMBERLAND,21502.0,...,72.0,142.0,92.0,143.0,91.0,2.1685,NaN,NaN,NaN,0
1,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,TOLEDO,43623.0,...,65.0,184.0,92.0,143.0,133.0,1.8029,NaN,NaN,NaN,0
2,1003000167,ESCOBAR,JULIO,E,DDS,M,I,5 PINE CONE RD,DAYTON,89403.0,...,72.0,16.0,17.0,NaN,NaN,1.0598,NaN,NaN,NaN,0
3,1003000175,REYES-VASQUEZ,BELINDA,NaN,D.D.S.,F,I,322 N AZUSA AVE STE 202,LA PUENTE,91744.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1003000282,BLAKEMORE,ROSIE,K,FNP,F,I,TENNESSEE PRISON FOR WOMEN,NASHVILLE,37243.0,...,62.0,NaN,NaN,NaN,NaN,4.5148,NaN,NaN,NaN,0


In [4]:
df.nunique()

npi                                 1162898
nppes_provider_last_org_name         262445
nppes_provider_first_name             82704
nppes_provider_mi                        34
nppes_credentials                     15790
nppes_provider_gender                     2
nppes_entity_code                         2
nppes_provider_street1               393764
nppes_provider_city                   12909
nppes_provider_zip5                   20266
nppes_provider_state                     61
nppes_provider_country                   41
specialty_description                   214
description_flag                          2
medicare_prvdr_enroll_status              3
total_claim_count                     21955
total_30_day_fill_count              139684
total_drug_cost                      927168
total_day_supply                     230328
bene_count                             2035
ge65_suppress_flag                        2
total_claim_count_ge65                19174
total_30_day_fill_count_ge65    

In [5]:
#Looking at country values
df['nppes_provider_country'].value_counts()

US    1162732
CA         33
DE         28
JP         11
KR         10
IN          9
MX          9
IT          7
SA          4
TR          4
PK          4
GB          4
TH          3
LB          3
IL          3
CN          2
AA          2
AE          2
KW          2
IE          2
ES          2
GR          2
QA          2
BA          1
KH          1
MW          1
CM          1
CL          1
BH          1
EG          1
NO          1
NL          1
NZ          1
JO          1
AG          1
UG          1
IQ          1
PE          1
AL          1
OM          1
AR          1
Name: nppes_provider_country, dtype: int64

In [6]:
#Classifying country by US and non US
df['country'] = df['nppes_provider_country'].apply(lambda x: 'US' if x == 'US' else 'Not US')

In [7]:
#Confirming transformation appears to have applied correctly
df['country'].value_counts()

US        1162732
Not US        166
Name: country, dtype: int64

In [8]:
#grouping together the three armed forces and the american territories in state column
df['state1'] = (df['nppes_provider_state'].str.replace('AA', 'AF').str.replace('AE', 'AF').str.replace('AP', 'AF'))

In [9]:
df['state'] = (df['state1'].str.replace('AS', 'AT').str.replace('GU', 'AT').str.replace('MP', 'AT').str.replace('PR', 'AT').str.replace('VI', 'AT'))

In [10]:
df['nppes_provider_state'].nunique()

61

In [11]:
#If conversions worked correctly, then the number of unique values for state column should be 55
df['state'].nunique()

55

In [12]:
#Dropping the non transformed country and state columns 
df1 = df.drop(columns = ['nppes_provider_state', 'nppes_provider_country', 'state1'])

In [13]:
df1.shape

(1162898, 76)

## Working with Null Values

All null values in the suppress fields is because that row is not suppressed. Therefore will change this is a categorical columns with null values filled in as 'not suppressed'

In [14]:
#Replacing null values for the suppress flag
df1.fillna({'bene_count_ge65_suppress_flag': 'not_suppressed', 'ge65_suppress_flag': 'not_suppressed', 'brand_suppress_flag': 'not_suppressed', 'generic_suppress_flag': 'not_suppressed', 'other_suppress_flag': 'not_suppressed', 'mapd_suppress_flag': 'not_suppressed', 'pdp_suppress_flag': 'not_suppressed', 'lis_suppress_flag': 'not_suppressed', 'nonlis_suppress_flag': 'not_suppressed'}, inplace=True)

In [15]:
#Creating a grouping for the suppress flag columns
suppress = ['bene_count_ge65_suppress_flag', 'ge65_suppress_flag', 'brand_suppress_flag', 'generic_suppress_flag', 'other_suppress_flag', 'mapd_suppress_flag', 'pdp_suppress_flag', 'lis_suppress_flag', 'nonlis_suppress_flag']

In [16]:
#Confirming no more null values exist for the suppress flag
df1[suppress].isnull().sum()

bene_count_ge65_suppress_flag    0
ge65_suppress_flag               0
brand_suppress_flag              0
generic_suppress_flag            0
other_suppress_flag              0
mapd_suppress_flag               0
pdp_suppress_flag                0
lis_suppress_flag                0
nonlis_suppress_flag             0
dtype: int64

In [17]:
#confirming three values now exist
df1['ge65_suppress_flag'].value_counts()

not_suppressed    910442
#                 204945
*                  47511
Name: ge65_suppress_flag, dtype: int64

In [21]:
#Saving cleaned file to new csv
df1.to_csv('Data/Providers_updated1.txt', sep='\t', index=False)